In [5]:
 #-*- coding: utf-8 -*-
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
import sys
def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
    else:
        print("lib path is already exists.")

importOwnLib()
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.feature_selection import RFECV, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import sonyalib as sonya
import keras
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout
# from tensorflow.python.keras._impl.keras import Model
# from tensorflow.python.keras._impl.keras.models import Model
from keras import losses
from keras.optimizers import Adam
import tensorflow as tf

lib path is already exists.


In [7]:
## Load dataset

PATH_EXCEL = './BRC2019_CRF_merged_200923 완료.xlsx'
metadata = sonya.get_metadata(PATH_EXCEL)
df_filtered = metadata.dropna(axis=0, how='any').reset_index(drop=True)  # NaN drop
properties = list(df_filtered.columns.values)
properties.remove('label')
# properties.remove('cN')
properties.remove('ID')
# X_origin = df_filtered['cN']
X_origin = df_filtered[properties]
y_origin = df_filtered['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_origin, y_origin, test_size=0.3, random_state=0)

In [9]:
## create pipeline 
## DecisionTreeClassifier
# create pipeline
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select = 5)
model = RandomForestClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

ValueError: could not convert string to float: 'R'

In [14]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

def mlp_model(num_input, dropout=0, lr=0.005, l1=9, l2=9):
    keras.backend.clear_session()
    ## 모델 구성하기
    model = Sequential()
    # print learning rate
    optimizer = Adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    model.add(Dense(l1, activation='relu', input_dim=num_input, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dense(l2, activation='relu', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))

    ## 모델 컴파일
    model.compile(optimizer=Adam(lr), loss=losses.binary_crossentropy, metrics=['accuracy', lr_metric])

    # model.summary()
    return model

In [15]:
[column for column in enumerate(properties)]

[(0, 'L/R'),
 (1, 'age'),
 (2, 'cT'),
 (3, 'aAverage'),
 (4, 'aSD'),
 (5, 'cAverage'),
 (6, 'cSD'),
 (7, 'LR'),
 (8, 'hu_diff'),
 (9, 'homogeneous'),
 (10, 'cN'),
 (11, 'hetero'),
 (12, 'rim'),
 (13, 'clustered'),
 (14, 'non-mass')]

## RFECV

In [17]:
print([column for column in enumerate(properties)])
svc = SVC(kernel="linear")
# rfe = RFECV(estimator=svc, step = 1, cv=StratifiedKFold(2), scoring="accuracy")
my_model = mlp_model(len(properties),lr=0.001, l1=9, l2=9)
rfecv = RFECV(estimator=DecisionTreeClassifier(), step = 1, cv=StratifiedKFold(5), scoring="accuracy")

rfecv.fit(X_train,y_train)
## rfe figure plot
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
fig = plt.gcf()
fig.savefig('/home/miruware/Pictures/ResearchDocuments/rfe.png')
plt.show()

## report which features were selected by RFE
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfecv.support_[i], rfecv.ranking_[i]))
    
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))    

[(0, 'L/R'), (1, 'age'), (2, 'cT'), (3, 'aAverage'), (4, 'aSD'), (5, 'cAverage'), (6, 'cSD'), (7, 'LR'), (8, 'hu_diff'), (9, 'homogeneous'), (10, 'cN'), (11, 'hetero'), (12, 'rim'), (13, 'clustered'), (14, 'non-mass')]


ValueError: could not convert string to float: 'R'

In [ ]:
# Fit the features to the response variable
rfecv.fit(X, y)

# Put the best features into new df X_new
X_new = rfecv.transform(X)


# X_new = rfecv.fit_transform(X, y)

In [ ]:
len(properties)

## RFE (BOXPLOT)

In [ ]:
model_input = tf.keras.Input(shape=(len(properties),))
layer_1 = tf.keras.layers.Dense(9, activation=tf.nn.relu)(model_input)
# Batch Normalization들어가야 함
layer_2 = tf.keras.layers.Dense(9, activation=tf.nn.relu)(layer_1)
model_output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(layer_2)
my_model2 = tf.keras.Model(inputs=model_input, outputs = model_output)
my_model2.compile()

In [ ]:
def optimizer_model(num_input, dropout=0, lr=0.005, l1=9, l2=9):
    
    keras.backend.clear_session()
    ## 모델 구성하기
    opt = Adam(lr=0.001, decay=1e-6)
    
    model_input = tf.keras.Input(shape=(len(properties),))
    layer_1 = tf.keras.layers.Dense(l1, activation=tf.nn.relu)(model_input)
    
    # Batch Normalization들어가야 함
    layer_2 = tf.keras.layers.Dense(l2, activation=tf.nn.relu)(layer_1)
    model_output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(layer_2)
    model = tf.keras.Model(inputs=model_input, outputs = model_output)
   
    model.compile(optimizer="Adam", loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model


X_train, X_test, y_train, y_test = train_test_split(X_origin, y_origin, test_size=0.3, random_state=0)
my_model = optimizer_model(len(properties),lr=0.005, l1=9, l2=7)
estimate_model = tf.keras.estimator.model_to_estimator(keras_model=my_model)


# results, names = list(), list()
# rfe = RFE(estimator=my_model, n_features_to_select=2)
# rfe.fit(X_train,y_train)

# print(X_train.columns[rfe.get_support(indices=True)].tolist())
# for nfeature in range(1, len(properties)):
#     rfe = RFE(extimator=my_model, n_features_to_select=nfeature)
    

In [ ]:

rfe = RFE(estimator=estimate_model, n_features_to_select=5)
rfe.fit(X_train, y_train)

## return the list of selected features
# X_train.columns[rfe.get_support(indices=True)].tolist()